In [2]:
# %load pyspark08_df.py
#!/usr/bin/env python


"""
@author:  'Administrator'
@contact:  
@time: 
"""

#!/usr/bin/python
# encoding: utf-8
# ================ 直接创建DataFrame
from pyspark.sql import SparkSession
from pyspark.sql import Row

spark = SparkSession.builder.appName('test').getOrCreate()
sc = spark.sparkContext

In [3]:
# spark.conf.set("spark.sql.shuffle.partitions", 6)
# ================直接创建==========================
l = [('Ankit',25),('Jalfaizy',22),('saurabh',20),('Bala',26)]
rdd = sc.parallelize(l)
people = rdd.map(lambda x: Row(name=x[0], age=int(x[1])))
# schemaPeople = sqlContext.createDataFrame(people)
schemaPeople = spark.createDataFrame(people)

In [4]:
schemaPeople.take(2)

[Row(age=25, name='Ankit'), Row(age=22, name='Jalfaizy')]

In [6]:
%pwd

'/home/ian/code/github/LSCJcourses/pysparkTJ'

In [7]:
# ==================从csv读取======================
df = spark.read.format("csv"). \
    option("header", "true") \
    .load("file:////home/ian/code/github/LSCJcourses/pysparkTJ/iris.csv")

df.printSchema()

root
 |-- SepalLength: string (nullable = true)
 |-- SepalWidth: string (nullable = true)
 |-- PetalLength: string (nullable = true)
 |-- PetalWidth: string (nullable = true)
 |-- Name: string (nullable = true)



In [8]:
df.show(10)

+-----------+----------+-----------+----------+-----------+
|SepalLength|SepalWidth|PetalLength|PetalWidth|       Name|
+-----------+----------+-----------+----------+-----------+
|        5.1|       3.5|        1.4|       0.2|Iris-setosa|
|        4.9|       3.0|        1.4|       0.2|Iris-setosa|
|        4.7|       3.2|        1.3|       0.2|Iris-setosa|
|        4.6|       3.1|        1.5|       0.2|Iris-setosa|
|        5.0|       3.6|        1.4|       0.2|Iris-setosa|
|        5.4|       3.9|        1.7|       0.4|Iris-setosa|
|        4.6|       3.4|        1.4|       0.3|Iris-setosa|
|        5.0|       3.4|        1.5|       0.2|Iris-setosa|
|        4.4|       2.9|        1.4|       0.2|Iris-setosa|
|        4.9|       3.1|        1.5|       0.1|Iris-setosa|
+-----------+----------+-----------+----------+-----------+
only showing top 10 rows



In [9]:
df.count()

150

In [12]:
df.columns

['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Name']

In [11]:
# ===============增加一列(或者替换) withColumn===========
# Column name which we want add /replace.
# Expression on column.

df.withColumn('newWidth',df.SepalWidth * 2).show(5)

+-----------+----------+-----------+----------+-----------+--------+
|SepalLength|SepalWidth|PetalLength|PetalWidth|       Name|newWidth|
+-----------+----------+-----------+----------+-----------+--------+
|        5.1|       3.5|        1.4|       0.2|Iris-setosa|     7.0|
|        4.9|       3.0|        1.4|       0.2|Iris-setosa|     6.0|
|        4.7|       3.2|        1.3|       0.2|Iris-setosa|     6.4|
|        4.6|       3.1|        1.5|       0.2|Iris-setosa|     6.2|
|        5.0|       3.6|        1.4|       0.2|Iris-setosa|     7.2|
+-----------+----------+-----------+----------+-----------+--------+
only showing top 5 rows



In [14]:
# ==========删除一列  drop=========================
df.drop('Name').show(5)

+-----------+----------+-----------+----------+
|SepalLength|SepalWidth|PetalLength|PetalWidth|
+-----------+----------+-----------+----------+
|        5.1|       3.5|        1.4|       0.2|
|        4.9|       3.0|        1.4|       0.2|
|        4.7|       3.2|        1.3|       0.2|
|        4.6|       3.1|        1.5|       0.2|
|        5.0|       3.6|        1.4|       0.2|
+-----------+----------+-----------+----------+
only showing top 5 rows



In [15]:
#================ 统计信息 describe================
df.describe().show()

+-------+------------------+-------------------+------------------+------------------+--------------+
|summary|       SepalLength|         SepalWidth|       PetalLength|        PetalWidth|          Name|
+-------+------------------+-------------------+------------------+------------------+--------------+
|  count|               150|                150|               150|               150|           150|
|   mean| 5.843333333333335| 3.0540000000000007|3.7586666666666693|1.1986666666666672|          null|
| stddev|0.8280661279778637|0.43359431136217375| 1.764420419952262|0.7631607417008414|          null|
|    min|               4.3|                2.0|               1.0|               0.1|   Iris-setosa|
|    max|               7.9|                4.4|               6.9|               2.5|Iris-virginica|
+-------+------------------+-------------------+------------------+------------------+--------------+



In [16]:
df.describe('Name').show()   #分类变量

+-------+--------------+
|summary|          Name|
+-------+--------------+
|  count|           150|
|   mean|          null|
| stddev|          null|
|    min|   Iris-setosa|
|    max|Iris-virginica|
+-------+--------------+



In [17]:
# ===============提取部分列 select==============
df.select('Name','SepalLength').show(5)

+-----------+-----------+
|       Name|SepalLength|
+-----------+-----------+
|Iris-setosa|        5.1|
|Iris-setosa|        4.9|
|Iris-setosa|        4.7|
|Iris-setosa|        4.6|
|Iris-setosa|        5.0|
+-----------+-----------+
only showing top 5 rows



In [18]:
# ==================基本统计功能 distinct count=====
df.select('Name').distinct().count()

3

In [19]:
# 分组统计 groupby(colname).agg({'col':'fun','col2':'fun2'})
df.groupby('Name').agg({'SepalWidth':'mean','SepalLength':'max'}).show()
# avg(), count(), countDistinct(), first(), kurtosis(): 峰度,
# max(), mean(), min(), skewness(), stddev(), stddev_pop(),
# stddev_samp(), sum(), sumDistinct(), var_pop(), var_samp() and variance()

+---------------+----------------+------------------+
|           Name|max(SepalLength)|   avg(SepalWidth)|
+---------------+----------------+------------------+
| Iris-virginica|             7.9|2.9739999999999998|
|    Iris-setosa|             5.8|3.4180000000000006|
|Iris-versicolor|             7.0|2.7700000000000005|
+---------------+----------------+------------------+



In [21]:
# 自定义的汇总方法

import pyspark.sql.functions as fn
df.agg(fn.count('SepalWidth').alias('width_count'),
       fn.countDistinct('Name').alias('distinct_Name_count')).collect()

[Row(width_count=150, distinct_Name_count=3)]

In [34]:
#====================数据集拆成两部分 randomSplit ===========
trainDF, testDF = df.randomSplit([0.6, 0.4])

可以看到，只是按照要求的比例近似拆分，而不是严格按照比例拆分（150的0.6是90）

In [35]:
df.count(),trainDF.count(),testDF.count()

(150, 100, 50)

In [36]:
# ================采样数据 sample===========
# withReplacement = True or False to select a observation with or without replacement.
# fraction = x, where x = .5 shows that we want to have 50% data in sample DataFrame.
# seed for reproduce the result
sdf = df.sample(False,0.2,100)#无放回抽样 抽20%，100是一个随机种子

In [37]:
sdf.count()

33

In [38]:
#查看两个数据集在类别上的差异 subtract，确保训练数据集覆盖了所有分类

diff_in_train_test = testDF.select('Name').subtract(trainDF.select('Name'))
diff_in_train_test.distinct().count()

0

In [39]:
diff_in_train_test.count()#0 说明训练集和测试集覆盖的分类无差异

0

In [40]:
#================== 综合案例，+ udf================
# 测试数据集中有些类别在训练集中是不存在的，把这些数据集应该从测试集中删除
trainDF,testDF = df.randomSplit([0.01,0.98])

diff_in_train_test = testDF.select('Name').subtract(trainDF.select('Name')).distinct().show()

+-----------+
|       Name|
+-----------+
|Iris-setosa|
+-----------+



上述说明测试集中有Iris-setosa这个分类，而训练集中没有。这会导致的一个直接后果就是：所训练的模型的预测结果中不会有Iris-setosa这个分类

In [41]:
# ================交叉表 crosstab=============
df.crosstab('Name','SepalLength').show()

+----------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|Name_SepalLength|4.3|4.4|4.5|4.6|4.7|4.8|4.9|5.0|5.1|5.2|5.3|5.4|5.5|5.6|5.7|5.8|5.9|6.0|6.1|6.2|6.3|6.4|6.5|6.6|6.7|6.8|6.9|7.0|7.1|7.2|7.3|7.4|7.6|7.7|7.9|
+----------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|  Iris-virginica|  0|  0|  0|  0|  0|  0|  1|  0|  0|  0|  0|  0|  0|  1|  1|  3|  1|  2|  2|  2|  6|  5|  4|  0|  5|  2|  3|  0|  1|  3|  1|  1|  1|  4|  1|
|     Iris-setosa|  1|  3|  1|  4|  2|  5|  4|  8|  8|  3|  1|  5|  2|  0|  2|  1|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|
| Iris-versicolor|  0|  0|  0|  0|  0|  0|  1|  2|  1|  1|  0|  1|  5|  5|  5|  3|  2|  4|  4|  2|  3|  2|  1|  2|  3|  1|  1|  1|  0|  0|  0|  0|  0|  0|  0|
+----------------+---+---+---+---+---+---+---+

In [43]:
# ===============sql 功能 ==============
df.registerTempTable('train_table')
spark.sql("select * from train_table limit 5").show()

+-----------+----------+-----------+----------+-----------+
|SepalLength|SepalWidth|PetalLength|PetalWidth|       Name|
+-----------+----------+-----------+----------+-----------+
|        5.1|       3.5|        1.4|       0.2|Iris-setosa|
|        4.9|       3.0|        1.4|       0.2|Iris-setosa|
|        4.7|       3.2|        1.3|       0.2|Iris-setosa|
|        4.6|       3.1|        1.5|       0.2|Iris-setosa|
|        5.0|       3.6|        1.4|       0.2|Iris-setosa|
+-----------+----------+-----------+----------+-----------+



In [44]:
#首先找到这些类，整理到一个列表

not_exist_cats = testDF.select('Name').subtract(trainDF.select('Name')).distinct().rdd.map(lambda x :x[0]).collect()

In [50]:
not_exist_cats

['Iris-setosa']

In [54]:
#定义一个方法，用于检测
def should_remove(x):
    if x in not_exist_cats:
        return "-1"
    else :
        return x

In [55]:
#创建udf，udf函数需要两个参数：
# Function
# Return type (in my case StringType())

from pyspark.sql.types import StringType
from pyspark.sql.functions import udf

check = udf(should_remove,StringType())

In [56]:
testDF.withColumn('New_name',check(testDF['Name'])).show()

+-----------+----------+-----------+----------+---------------+---------------+
|SepalLength|SepalWidth|PetalLength|PetalWidth|           Name|       New_name|
+-----------+----------+-----------+----------+---------------+---------------+
|        4.3|       3.0|        1.1|       0.1|    Iris-setosa|             -1|
|        4.4|       2.9|        1.4|       0.2|    Iris-setosa|             -1|
|        4.4|       3.0|        1.3|       0.2|    Iris-setosa|             -1|
|        4.4|       3.2|        1.3|       0.2|    Iris-setosa|             -1|
|        4.5|       2.3|        1.3|       0.3|    Iris-setosa|             -1|
|        4.6|       3.1|        1.5|       0.2|    Iris-setosa|             -1|
|        4.6|       3.2|        1.4|       0.2|    Iris-setosa|             -1|
|        4.6|       3.4|        1.4|       0.3|    Iris-setosa|             -1|
|        4.6|       3.6|        1.0|       0.2|    Iris-setosa|             -1|
|        4.7|       3.2|        1.3|    

In [57]:
testDF2 = testDF.withColumn('New_name',check(testDF['Name'])).filter('New_name <> "-1"')

testDF2.show()

+-----------+----------+-----------+----------+---------------+---------------+
|SepalLength|SepalWidth|PetalLength|PetalWidth|           Name|       New_name|
+-----------+----------+-----------+----------+---------------+---------------+
|        4.9|       2.4|        3.3|       1.0|Iris-versicolor|Iris-versicolor|
|        4.9|       2.5|        4.5|       1.7| Iris-virginica| Iris-virginica|
|        5.0|       2.0|        3.5|       1.0|Iris-versicolor|Iris-versicolor|
|        5.0|       2.3|        3.3|       1.0|Iris-versicolor|Iris-versicolor|
|        5.1|       2.5|        3.0|       1.1|Iris-versicolor|Iris-versicolor|
|        5.2|       2.7|        3.9|       1.4|Iris-versicolor|Iris-versicolor|
|        5.4|       3.0|        4.5|       1.5|Iris-versicolor|Iris-versicolor|
|        5.5|       2.3|        4.0|       1.3|Iris-versicolor|Iris-versicolor|
|        5.5|       2.4|        3.7|       1.0|Iris-versicolor|Iris-versicolor|
|        5.5|       2.4|        3.8|    

In [58]:
testDF2.select('Name').distinct().show()

+---------------+
|           Name|
+---------------+
| Iris-virginica|
|Iris-versicolor|
+---------------+



In [65]:
# ==================过滤行 filter ==================
estDF2 = df.withColumn('New_name',check(testDF['Name'])).filter('New_name <> "-1"')

In [66]:
estDF2.show()

+-----------+----------+-----------+----------+---------------+---------------+
|SepalLength|SepalWidth|PetalLength|PetalWidth|           Name|       New_name|
+-----------+----------+-----------+----------+---------------+---------------+
|        7.0|       3.2|        4.7|       1.4|Iris-versicolor|Iris-versicolor|
|        6.4|       3.2|        4.5|       1.5|Iris-versicolor|Iris-versicolor|
|        6.9|       3.1|        4.9|       1.5|Iris-versicolor|Iris-versicolor|
|        5.5|       2.3|        4.0|       1.3|Iris-versicolor|Iris-versicolor|
|        6.5|       2.8|        4.6|       1.5|Iris-versicolor|Iris-versicolor|
|        5.7|       2.8|        4.5|       1.3|Iris-versicolor|Iris-versicolor|
|        6.3|       3.3|        4.7|       1.6|Iris-versicolor|Iris-versicolor|
|        4.9|       2.4|        3.3|       1.0|Iris-versicolor|Iris-versicolor|
|        6.6|       2.9|        4.6|       1.3|Iris-versicolor|Iris-versicolor|
|        5.2|       2.7|        3.9|    

In [64]:
import pyspark.sql.functions as fun
df.withColumn('New_name',check(testDF['Name'])).filter(fun.col('New_name')!="-1").show()

+-----------+----------+-----------+----------+---------------+---------------+
|SepalLength|SepalWidth|PetalLength|PetalWidth|           Name|       New_name|
+-----------+----------+-----------+----------+---------------+---------------+
|        7.0|       3.2|        4.7|       1.4|Iris-versicolor|Iris-versicolor|
|        6.4|       3.2|        4.5|       1.5|Iris-versicolor|Iris-versicolor|
|        6.9|       3.1|        4.9|       1.5|Iris-versicolor|Iris-versicolor|
|        5.5|       2.3|        4.0|       1.3|Iris-versicolor|Iris-versicolor|
|        6.5|       2.8|        4.6|       1.5|Iris-versicolor|Iris-versicolor|
|        5.7|       2.8|        4.5|       1.3|Iris-versicolor|Iris-versicolor|
|        6.3|       3.3|        4.7|       1.6|Iris-versicolor|Iris-versicolor|
|        4.9|       2.4|        3.3|       1.0|Iris-versicolor|Iris-versicolor|
|        6.6|       2.9|        4.6|       1.3|Iris-versicolor|Iris-versicolor|
|        5.2|       2.7|        3.9|    